In [4]:
import os
import openai
from openai.error import RateLimitError
from dotenv import load_dotenv
import time

load_dotenv()

def get_secret(name):
    if name in os.environ:
        return os.environ[name]
    return os.getenv(name)

# create .env in project dir and set api keys there
openai.api_key = get_secret('OPENAI_API_KEY')

In [5]:
# for Postman experiment we insert "\n" as like this(json strings break because of newlines):
# uint256 length = 10;\nfunction loop() public {\n    for (uint256 i = 0; i < length; i++) {\n        // do something here\n    }\n}
prompt_str = """
Gas optimize the following solidity code:
function loop(uint[] memory arr) external pure returns (uint sum) {
    for (uint i = 0; i < arr.length; i++) {
        sum += arr[i];
    }
}
"""

In [6]:
completion_api_parameters = {
  'temperature': 0.2,
  'model': 'gpt-3.5-turbo-0301',
  'max_tokens': 512
}

for _ in range(3):
    try:
        completion = openai.ChatCompletion.create(**completion_api_parameters,
                                                  messages=[{"role": "user", "content": prompt_str}])
        print(completion.choices[0].message.content.strip(" \n"))
        break
    except RateLimitError:
        time.sleep(10)

function loop(uint[] memory arr) external pure returns (uint sum) {
    uint len = arr.length;
    for (uint i = 0; i < len; i++) {
        sum += arr[i];
    }
} 

Explanation: 

- We can optimize the gas cost of the loop by storing the length of the array in a variable before the loop starts. This way, the loop condition does not need to access the length of the array on every iteration, which saves gas. 
- We can declare the variable `len` as `uint` to save gas, since `uint` is the default integer type in Solidity. 
- We can remove the `memory` keyword from the function parameter, since it is not necessary for a pure function.


In [7]:

examples = """ORIGINAL:
*
function test() external returns (uint256) { varr += block.number; return varr; }
*
OPTIMIZED:
function test() external returns (uint256) {
varr = varr + block.number;
return varr;
}
ORIGINAL:
*
function execution() external {    for (uint i; i < 5; i+=1)    {} }
*
OPTIMIZED:
function execution() external {
   for (uint i; i < 5; ++i)
   {}
}
ORIGINAL:
*
function execution() external {    for (uint i; i < 5; i++)    {} }
*
OPTIMIZED:
function execution() external {
   for (uint i; i < 5; ++i)
   {}
}
ORIGINAL:
*
function execution() external returns (uint) { uint _num = 5; return _num; }
*
OPTIMIZED:
function execution() external returns (uint _num) {
_num = 5;
}
ORIGINAL:
*
for (uint i = 0; i < 5; ++i) {}
*
OPTIMIZED:
for (uint i; i < 5; ++i) {}"""

example_to_complete = """ORIGINAL:
```
function loop(uint[] memory arr) external pure returns (uint sum) {
    for (uint i = 0; i < arr.length; i++) {
        sum += arr[i];
    }
}
```
OPTIMIZED:"""

prompt_str = """In this task, you need to rewrite solidity code surrounded 
by triple backticks with the operations that consume less gas while 
implementing the same logic.\n
Guidelines:\n
- Read state variables as few times as possible in a function.\n
- Write state variables as few times as possible in a function.\n
- ked arithmetic operations if underflow and overflow will
never happen.\n
- Use immutable variables if their values are never
changed outside the constructor.\n
- For every storage read, check that
it is done only once in a function.\n
- For every storage write, check
that it is done only once in a function.\n
- For every storage variable,
check that it's used.\n
- For every storage variable, check if it would be
cheaper to pack it together with other variables. Check if it could fit
in a smaller type.\n
- Check that the assigned value is used after the
assignment for every local variable.\n
- For every public function, check
- In class declarations order small values sequentially to use less storage space
that they could be made as external.\n{}{}""".format(examples, example_to_complete)


for _ in range(3):
    try:
        completion = openai.ChatCompletion.create(**completion_api_parameters,
                                                  messages=[{"role": "user", "content": prompt_str}])
        print(completion.choices[0].message.content.strip(" \n"))
        break
    except RateLimitError:
        time.sleep(10)

function loop(uint[] memory arr) external pure returns (uint sum) {
    uint arrLength = arr.length;
    for (uint i = 0; i < arrLength; ++i) {
        sum += arr[i];
    }
}
